#### Copyright 2019 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Android Management API - Quickstart for Web Apps

This notebook shows you how to create and manage web apps with the Android Management API. 
It is assumed that you have worked through the [Android Management API quickstart guide](https://developers.google.com/android/management/quickstart) and are therefore familiar with how to use this colab notebook and already have a cloud project, test enterprise, and provisioned device.


## Authenticating
To create and access resources, you need to authenticate with an account that has edit rights over your project. To start the authentication flow, **run the cell below**.

In [0]:
import json
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import pprint

# This is a public OAuth config that you can use to run this guide.
# However, use different credentials when building your own solution.
CLIENT_CONFIG = {
    'installed': {
        'client_id':'882252295571-uvkkfelq073vq73bbq9cmr0rn8bt80ee.apps.googleusercontent.com',
        'client_secret': 'S2QcoBe0jxNLUoqnpeksCLxI',
        'auth_uri':'https://accounts.google.com/o/oauth2/auth',
        'token_uri':'https://accounts.google.com/o/oauth2/token'
    }
}
SCOPES = ['https://www.googleapis.com/auth/androidmanagement']

# Run the OAuth flow.
flow = InstalledAppFlow.from_client_config(CLIENT_CONFIG, SCOPES)
credentials = flow.run_console()

# Create the API client.
androidmanagement = build('androidmanagement', 'v1', credentials=credentials)

print('\nAuthentication succeeded.')

# To improve the readability of deeply nested data structures, we create a 
# helper function to pretty print a result:
prettyprint = pprint.PrettyPrinter(indent=4).pprint


## Declare your enterprise
In the [general Quickstart](https://developers.google.com/android/management/quickstart) you already created a test enterprise which will be used here.

In [0]:
# Paste your enterprise name here.
# It should be of the format 'enterprises/LC...'
enterprise_name = ''

# Create a web app with the managed Google Play iframe

The managed Google Play iframe includes a user interface for IT admins to create, edit, and delete web apps.

**Run the cell below** to open the managed Google Play iframe.

In the iframe:
*   In the left navigation menu click **Web apps**
*   Click the **+** button
*   Enter a title and a URL, select a display mode, and optionally upload an icon 
*   Click **Create**
*   In the web app list click on the web app you've just created
*   Click **Select**
*   Copy the package name of the web app and add it in the policy below



In [0]:
IFRAME_URL = "https://storage.googleapis.com/android-management-api-samples/managed_play_iframe.html"

web_token = androidmanagement.enterprises().webTokens().create(
    parent=enterprise_name,
    body={
      "parentFrameUrl": IFRAME_URL
    }
).execute()

print('Open the managed Google Play iframe:', IFRAME_URL + "?token=" + web_token["value"])

# Distributing the web app

Once a web app is created, it can be used just like a normal app, e.g. added to a policy. Assuming you already used the policy `/policies/policy1` from the [Android Management API quickstart guide](https://developers.google.com/android/management/quickstart) when you set up the test device, we will use that name.

**Important**: For a device to support web apps, Google Chrome must be installed. When distributing web apps, ensure that Google Chrome (`com.android.chrome`) is always included in a device's policy.



In [0]:
policy_name = enterprise_name + '/policies/policy1'

# Paste the package name of the web app here
# It should be of the format 'com.google.enterprise.webapp.*'.
web_app_package_name = ''

policy = {
    'applications': [
        {
            'installType': 'FORCE_INSTALLED',
            'packageName': 'com.android.chrome'
        },
        {
            'installType': 'FORCE_INSTALLED',
            'packageName': web_app_package_name
        }
    ],
    'debuggingFeaturesAllowed': True
}

# Write the new policy:
result = androidmanagement.enterprises().policies().patch(
    name=policy_name,
    body=policy
).execute()

prettyprint(result)

# Create a web app with the Android Management API

Another way to support web apps is to use the Android Management API to create and manage web apps.

## enterprises.webApps.create

You can use the method [`enterprises.webApps.create`](https://developers.google.com/android/management/reference/rest/v1/enterprises.webApps/create) to create a web app. You need just a title and a URL, plus the display mode (can be one of `MINIMAL_UI`, `STANDALONE`, or `FULL_SCREEN`).

While an icon is optional, it's best practice to always specify a meaningful icon for a web app. Like a title, an icon helps users identify an app. Google applies the same fixed static icon for all web apps that don't have a specified icon.

The icon data needs to be encoded in base64url (see [RFC 4648, section-5](https://tools.ietf.org/html/rfc4648#section-5)). It should be a square (or near-square) shape, ideally around 512x512 pixels.

In [0]:
import base64
import urllib.request

# A useful picture found on the Wikipedia page:
icon_url = 'https://developer.android.com/images/brand/Android_Robot.png'

icon_raw_data = urllib.request.urlopen(icon_url).read()
icon_encoded_data = base64.urlsafe_b64encode(icon_raw_data).decode("utf-8")

web_app = {
    "title": "Android",
    "startUrl": "https://en.m.wikipedia.org/wiki/Android_(operating_system)",
    "displayMode": "MINIMAL_UI",
    "icons": [
        {
            "imageData": icon_encoded_data
        }
    ]
}

result = androidmanagement.enterprises().webApps().create(
    parent=enterprise_name,
    body=web_app
).execute()

# Take note of the app's name (which is it's id) for future operations:
web_app_name = result['name']

prettyprint(result)

The result of the create call basically mirrors its inputs but with the icon data and the name filled in.

For later use, the `name` component of the result is the imporant piece of information, which should be extracted. A web app's package name is the last component of its name, i.e. the part after the last slash (`/`) character.

## enterprises.webApps.patch

With the `enterprises.webApps.patch` call you can update any aspect of a web app.

E.g. here we change the title and display mode.

In [0]:
web_app = {
    "title": "Wikipedia: Android",
    "startUrl": "https://en.m.wikipedia.org/wiki/Android_(operating_system)",
    "displayMode": "STANDALONE",
    "icons": [
        {
            "imageData": icon_encoded_data
        }
    ]
}

result = androidmanagement.enterprises().webApps().patch(
    name=web_app_name,
    body=web_app
).execute()

prettyprint(result)

## enterprises.webApps.list

The `enterprises.webApps.list` method returns information about all web apps. 

This is also a handy way to recover the name (aka the id) of a web app should it be forgotten after creation.

In [0]:
webapps = androidmanagement.enterprises().webApps().list(
    parent=enterprise_name
).execute()

prettyprint(webapps)

## enterprises.webApps.get

To get details about a single web app, the `enterprises.webApps.get` call is used.

*Note* that data from the list call does not contain information about the icons. To get that, the get is is needed.

In [0]:
webapp = androidmanagement.enterprises().webApps().get(
    name=web_app_name
).execute()

prettyprint(webapp)

## enterprises.webApps.delete

To delete a web app, call `enterprises.webApps.delete`.

*Note* that deleting the web app does not uninstall it from devices it's installed on.

In [0]:
androidmanagement.enterprises().webApps().delete(
    name=web_app_name
).execute()

# Web apps in the Play store

Web apps are private apps, and are therefore only visible to the IT admins and users of the enterprise for which they are created. After logging into managed Google Play, IT admins can view the full store listing of a web app  at `https://play.google.com/work/apps/details?id=<packagename>`.

For users web apps are just like regular Android apps and can see them in the Work store on their device.


In [0]:
print ("Admins visit https://play.google.com/work/apps/details?id={}, users https://play.google.com/store/apps/details?id={} to see the Play store listing.".format(web_app_package_name, web_app_package_name))